In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
from bs4 import BeautifulSoup
#from itertools import chain
import requests
import time
import datetime

import smtplib
#import re

print('Setup Complete')

Setup Complete


In [4]:
URL = 'http://books.toscrape.com/catalogue/category/books_1/index.html'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 OPR/95.0.0.0"}

page = requests.get(URL, headers=headers)

soup1 = BeautifulSoup(page.content, "lxml")

soup2 = BeautifulSoup(soup1.prettify(), "lxml")
    

In [5]:
#scraping all the genres
genres = []
aside = soup2.find('aside')
a = aside.find_all('a')
for j in a:
    href = j.attrs['href']
    genre = href.strip('./')
    genre = href.strip('/index.html')
    genres.append(genre)
genres.remove('')



In [25]:
#scraping genre names
genre_name = []
aside = soup2.find('aside')
a = aside.find_all('a')
for j in a:
    name = j.get_text(strip=True)
    genre_name.append(name)
    
genre_name.remove('Books')
    



In [13]:
#finding number of books per genre
nr_books = []
for g in genres:
    URL = f'http://books.toscrape.com/catalogue/category/{g}/index.html'
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 OPR/95.0.0.0"}

    page = requests.get(URL, headers=headers)

    soup1 = BeautifulSoup(page.content, "lxml")

    soup2 = BeautifulSoup(soup1.prettify(), "lxml")
    
    form = soup2.find('form')
    strong = form.find('strong').get_text(strip=True)
    nr_books.append(strong)
nr_books = [float(x) for x in nr_books]
#nr_books

In [15]:
#finding number of result pages for each genre
import math as math
pages = []
for nr in nr_books:
    pgs = math.ceil(nr/20)
    pages.append(pgs)


In [ ]:
for genre in genres:
    pg=pages[genres.index(genre)]
    if (pg>1):
        for i in range(1,pg+1):
            URL = f'http://books.toscrape.com/catalogue/category/{genre}/page-{i}.html'
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 OPR/95.0.0.0"}

            page = requests.get(URL, headers=headers)

            soup1 = BeautifulSoup(page.content, "lxml")

            soup2 = BeautifulSoup(soup1.prettify(), "lxml")

            ol = soup2.find('ol')
            print(ol)
                #articles = ol.find_all('article', class_='product_pod')

In [76]:
#let's try to iterate this shit
books = []
for genre in genres:
    pg=pages[genres.index(genre)]
    if (pg==1):
        URL = f'http://books.toscrape.com/catalogue/category/{genre}/index.html'
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 OPR/95.0.0.0"}

        page = requests.get(URL, headers=headers)

        soup1 = BeautifulSoup(page.content, "lxml")

        soup2 = BeautifulSoup(soup1.prettify(), "lxml")
#****scraping book title, rating, price****
        ol = soup2.find('ol')


        articles = ol.find_all('article', class_='product_pod')


        for article in articles:
            image = article.find('img')

            title = image.attrs['alt']

            starTag = article.find('p')

            star = starTag['class'][1]
            price = article.find('p', class_='price_color').get_text(strip= True)
            price = price.strip('£')
            gnr = genre_name[genres.index(genre)]

            books.append([title, gnr, star, price])

        
    else:
        for i in range(1,pg+1):
            URL = f'http://books.toscrape.com/catalogue/category/{genre}/page-{i}.html'
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36 OPR/95.0.0.0"}

            page = requests.get(URL, headers=headers)

            soup1 = BeautifulSoup(page.content, "lxml")

            soup2 = BeautifulSoup(soup1.prettify(), "lxml")

            ol = soup2.find('ol')

            articles = ol.find_all('article', class_='product_pod')

            for article in articles:
                image = article.find('img')

                title = image.attrs['alt']

                starTag = article.find('p')

                star = starTag['class'][1]

                price = article.find('p', class_='price_color').get_text(strip= True)
                price = price.strip('£')
                gnr = genre_name[genres.index(genre)]
                books.append([title, gnr, star, price])
        

In [77]:
len(books)

1000

In [78]:
df = pd.DataFrame(books, columns=['Title', 'Genre', 'Rating', 'Price'])
df.to_csv('all_books_scraped.csv')
data=pd.read_csv('/kaggle/working/all_books_scraped.csv')

In [83]:
dict = {'One':'1','Two':'2','Three':'3','Four':'4','Five':'5'}

data.replace({'Rating':dict},inplace=True)

In [90]:
data.Rating.astype('int64')

0      2
1      4
2      3
3      2
4      3
      ..
995    4
996    1
997    1
998    5
999    1
Name: Rating, Length: 1000, dtype: int64

In [93]:
data.Genre.value_counts()

Default               152
Nonfiction            110
Sequential Art         75
Add a comment          67
Fiction                65
Young Adult            54
Fantasy                48
Romance                35
Mystery                32
Food and Drink         30
Childrens              29
Historical Fiction     26
Poetry                 19
Classics               19
History                18
Womens Fiction         17
Horror                 17
Science Fiction        16
Science                14
Music                  13
Business               12
Travel                 11
Thriller               11
Philosophy             11
Humor                  10
Autobiography           9
Art                     8
Religion                7
Psychology              7
Spirituality            6
New Adult               6
Christian Fiction       6
Self Help               5
Sports and Games        5
Biography               5
Health                  4
Contemporary            3
Christian               3
Politics    

In [97]:
data.drop(data[data['Genre']=='Add a comment'].index, inplace=True)

In [98]:
data.to_csv('all_books_scraped.csv')